In [1]:
import pandas as pd
import numpy as np 
import json
import requests 
from config import *

In [2]:
df_main = pd.read_csv(csv_file_path)
#drop the duplicate transaction in case there is any
df_main = df_main.drop_duplicates(subset=[' user_tx'])

In [198]:
#make a list of transaction hash we need to analyse
tx_hash_list = [x for x in df_main[' user_tx'].to_list() if pd.notnull(x)]

# Infuria
With this API, we want to get all the inputs necessary to simulate the transaction again later on. Infuria gives us all of these inputs except fot the timestamp of the transaction, which is why we need to use the Etherscan API later on.

In [199]:
#the api key is stored in the config file
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

#Get the infuria response for the first transaction in the list to create a dataframe
payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionByHash",
  "params": [tx_hash_list[0]],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dct = response.json()['result']
dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
df_infuria = pd.DataFrame(dct, index=[0])

# get the infuria response for all the other transactions in the list and append the results to the above dataframe
for tx_hash in tx_hash_list[1:]:
  payload = json.dumps({
    "jsonrpc": "2.0",
    "method": "eth_getTransactionByHash",
    "params": [tx_hash],
    "id": 1
  })
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  if response.ok:
    dct = response.json()['result']
    dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct, index=[0])
    df_infuria = pd.concat([df_infuria, df_temp])
    
  else: 
    print(f"error code {response.status_code} for transaction {tx_hash}")


In [200]:
#merge infuria response to main
df_main = df_main.merge(df_infuria, left_on = ' user_tx', right_on = 'hash', how ='outer')

In [201]:
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity'],
      dtype='object')

# Etherscan 

We use this API to have the timestamp of the blocks rather than the transactions themselves to reduce the amount of API calls (tx and block time are the same for all tx in the block). We need the timestamp of the transactions because in the Tenderly API, if we do not override the timestamp, then it uses the current time as input variable

In [202]:
#getting all the block numbers of the transactions we want to analyse
block_number_list = list(set([x for x in df_main['block_number'].to_list() if pd.notnull(x)]))

In [203]:
# Get the ehterscan response for the first block in the list to create a dataframe
url_eth = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number_list[0]}&apikey={eth_scan_api_key}"

response_eth = requests.request("POST", url_eth)

dct_eth = response_eth.json()['result']
dct_eth = {k: None if not v else v for k, v in dct_eth.items()} # making sure none of the values are empty
df_eth = pd.DataFrame(dct_eth, index=[0])

# Get the etherscan responses for the other blocks in the list
for block in block_number_list[1:]:
  url_temp = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block}&apikey={eth_scan_api_key}"
  response_temp = requests.request("POST", url_temp)

  if response_temp.ok:
    dct_temp = response_temp.json()['result']
    dct_temp = {k: None if not v else v for k, v in dct_temp.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct_temp, index=[0])
    df_eth = pd.concat([df_eth, df_temp])
    
  else: 
    print(f"error code {response_temp.status_code} for block {block}")

In [204]:
df_eth['blockNumber'] = df_eth['blockNumber'].astype(int)

In [205]:
#merge eth to main
df_main = df_main.merge(df_eth, left_on = 'block_number', right_on = 'blockNumber', how ='outer')

# Tenderly
Here we finally do the simulation. We do it once at the original index position to get the amount of coin transferred originally. Then we do it again at index position 0.

In [206]:
headers = {
    'X-Access-Key': f'{tenderly_access_token}',
    'content-type': 'application/json',
}

#creating an empty DataFrame for the results
columns = ['tx_hash', 'index', 'type', 'raw_amount', 'dollar_value', 'token_contract_address', 'token_name', 'token_dollar_value']
df_results = pd.DataFrame(columns = columns)

# creating a list of tx hashes where the tenderly api returned nothing
tx_hash_problem_list = []

#iterating over every row of the main dataframe (one row is one transaction)
for index, row in df_main.iterrows():
    tx_index_list = [0]
    tx_index_list.append(int(row['transactionIndex'], 0))
    # for each transaction, simulate twice: once for each index
    for tx_index in tx_index_list:
        json_data = {
        'network_id': int(row['chainId'], 0),
        'from': row['from'],
        'to': row['to'],
        'input': row['input'],
        'block_number': row['block_number'],
        'transaction_index': tx_index,
        'simulation_type': 'quick',
        'gas': int(row['gas'], 0),
        'value': int(row['value'], 0),
        'gas_price': int(row['gasPrice'], 0),
        'l1_timestamp': int(row['timeStamp'])
        }
        
        response = requests.post(
        'https://api.tenderly.co/api/v1/account/aurelie2/project/cowswap2/simulate',
        headers=headers,
        json=json_data,
        )

        try:
            for data in response.json()['transaction']['transaction_info']['asset_changes']:
                tx_type = data['type']
                tx_raw_amount = data['raw_amount']
                tx_dollar_value = data['dollar_value']

                #sometimes the following values are empty 
                try:
                    contract_address = data['token_info']['contract_address']
                except:
                    contract_address = 'None'

                try:
                    token_name = data['token_info']['name']
                except:
                    token_name = 'None'
                try:
                    token_dollar_value = data['token_info']['dollar_value']
                except:
                    token_dollar_value = 'None'

                new_row = {
                    'tx_hash' : row['hash'],
                    'index' : tx_index, 
                    'type': tx_type, 
                    'raw_amount': tx_raw_amount, 
                    'dollar_value' : tx_dollar_value, 
                    'token_contract_address': contract_address, 
                    'token_name': token_name, 
                    'token_dollar_value': token_dollar_value
                    }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except:
            example = response.json()
            #tx_hash_problem = df_main[df_main['input'] == row['input']]['hash'].values[0]
            tx_hash_problem_list.append(row['hash'])
            


In [207]:
#creating a new dataframe with only the transactions where tenderly api returned something.This meanse that there are 2 index values for these good transactions.
grouped = df_results.groupby('tx_hash')
df_results_good = grouped.filter(lambda x: x['index'].nunique() == 2)

In [208]:
print(
    'number of transactions for which this approach worked:', df_results_good.tx_hash.nunique(),
    "\nnumber of transactions for which this approach did not worked:", len(set(tx_hash_problem_list)),
    '\npercentage of properly simulated transactions:', format(df_results_good.tx_hash.nunique()/(df_results_good.tx_hash.nunique() + len(set(tx_hash_problem_list))), ".2%")
    )


number of transactions for which this approach worked: 120 
number of transactions for which this approach did not worked: 10 
percentage of properly simulated transactions: 92.31%


# Calculate the dollar value of the potential loss for the coins that don't have a dollar value
The logic here is 
- first calculate the difference in dollar if the simulation gives it to us: dollar_value_diff
- if we do not have any dollar value for a coin, there will always be a WETH value in the transaction. So we will get the value of the unknown coin based on the WETH value: calulated_dollar_diff
- we get the loss for that transaction (dollar_diff) by chosing the dollar_value_diff (if exists) or calulated_dollar_diff (if dollar_value_diff does not exist) 
- we sum the dollar_diff to calculte the total loss over all transaction

In [209]:
# when the coin appears multiple times, then the amounts are the same anyway
df_results_good = df_results_good.groupby(['tx_hash', 'index', 'token_contract_address'], as_index=False).first()
df_results_good['dollar_value'] = df_results_good['dollar_value'].astype(float)
df_results_good['dollar_value_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['dollar_value'].diff()

In [210]:
#first if token name is WETH, then the raw_amount need tos be converted. Every transaction has WETH so I will base the analysis on that 
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good.loc[df_results_good['token_name'] == 'WETH', 'raw_amount'] /= 1e+18

In [211]:
def calculate_dollar_value(row, df):
    if pd.isna(row['dollar_value']):
        # Filter the DataFrame to get the subset with the same 'tx_hash' and 'index' values
        subset = df[(df['tx_hash'] == row['tx_hash']) & (df['index'] == row['index'])]

        if not subset.empty:
            # Calculate dollar value based on 'WETH' value if the subset is not empty
            weth_row = subset[subset['token_name'] == 'WETH']
            if not weth_row.empty:
                weth_dollar_value = float(weth_row['dollar_value'].values[0])
                unknown_raw_amount = float(row['raw_amount'])
                return weth_dollar_value / unknown_raw_amount

    # If token_name is not NaN or if the subset is empty, return the original dollar_value
    return row['dollar_value']


In [212]:

# Apply the function to each row
# Iterate over each row in the DataFrame
for index, row in df_results_good.iterrows():
    # Call the calculate_dollar_value function and pass the current row and the DataFrame
    calculated_value = calculate_dollar_value(row, df_results_good)
    
    # Update the 'dollar_value' column with the calculated value
    df_results_good.at[index, 'dollar_value'] = calculated_value


In [213]:
df_results_good = df_results_good.sort_values(by = ['tx_hash', 'index'], ascending = [True, False])

In [214]:
# FYI if token_contract_address == 'None', then it's always Ethereum
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good['raw_amount_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['raw_amount'].transform(lambda x: x[::-1].diff())
df_results_good['calulated_dollar_diff'] = df_results_good['raw_amount_diff'] * df_results_good['dollar_value']
df_results_good['dollar_diff'] = np.where(pd.notnull(df_results_good['dollar_value_diff']), df_results_good['dollar_value_diff'], df_results_good['calulated_dollar_diff'])

In [215]:
print("this is the total potential loss in dollars for the given transactions", df_results_good.dollar_diff.sum())

this is the total potential loss in dollars for the given transactions -495764.49344588537


# export the file

In [216]:
# drop these columns to avoid confusion since the dollar_diff column encompasses these values already
df_results_good.drop(columns=['calulated_dollar_diff', 'dollar_value_diff'], inplace=True)

In [217]:
df_results_good.to_csv('results.csv')